In [15]:
from random import Random
from scapy.all import *
from Modbus.Modbus import *

srcIp = '192.168.10.206'
srcPort = random.randint(1024, 65535)
destIp = '192.168.10.211'
destPort = 502
seqNumber = random.randint(444, 8765432)
ackNr = 0
transId = random.randint(44, 44444)

def updateSeqAndAckNrs(sendPkt, recvPkt):
    global seqNumber
    global ackNr
    seqNumber = seqNumber + len(sendPkt [TCP].payload)
    ackNr = ackNr + len(recvPkt [TCP].payload)

def sendAck():
    ip = IP(src=srcIp, dest=destIp)
    ACK = TCP(sport=srcPort, dport=destPort, flags='A', seq=seqNumber, ack=ackNr)
    pktACK = ip / ACK
    send(pktACK)

def tcpHandshake():
    global seqNumber
    global ackNr

    ip = IP(src=srcIp, dest=destIp)
    SYN = TCP(sport=srcPort, dport=destPort, flags='S', seq=seqNumber, ack=ackNr)
    pktSYN = ip / SYN

    pktSYNACK = sr1(pktSYN)

    ackNr = pktSYNACK.seq + 1
    seqNumber = seqNumber + 1
    ACK = TCP(sport=srcPort, dport=destPort, flags='A', seq=seqNumber, ackNr=ackNr)

    send(ip/ACK)
    return ip/ACK

def endConnection():
    ip = IP(srcIp, dest=destIp)
    RST = TCP(sport=srcPort, dport=destPort, flags='RA', seq=seqNumber, ackNr=ackNr)
    pktRST = ip / RST
    send(pktRST)

def connectedSend(pkt):
    pkt[TCP].flags = 'PA'
    pkt[TCP].seq = seqNumber
    pkt[TCP].ack = ackNr
    send(pkt)

connectionPkt = tcpHandshake()

ModbusPkt = connectionPkt/ModbusADU()/ModbusPDU01_Read_Coils()

ModbusPkt[ModbusADU].uintId = 1

ModbusPkt[ModbusPDU01_Read_Coils].funcCode = 1
ModbusPkt[ModbusPDU01_Read_Coils].quantity = 5


for i in range(1, 6):
    ModbusPkt[ModbusADU].transId = transId + i*3
    ModbusPkt[ModbusPDU01_Read_Coils].startAddr = random.randint(0, 65535)

    connectedSend(ModbusPkt)

    Results = sniff(count=1, filter='tcp[tcpflags] & (tcp-push|tcp-ack) != 0')

    ResponsePkt = Results[0]
    updateSeqAndAckNrs(ModbusPkt, ResponsePkt)
    ResponsePkt.show()

endConnection()

AttributeError: dest